# 注意
使用colab時請記得將執行階段類型設為GPU

In [ ]:
! git clone https://github.com/NCHU-NLP-Lab/demeterchain.git
! pip install demeterchain

In [ ]:
cd demeterchain/examples

# 讀取與分割文件

In [2]:
from demeterchain.loaders import TextLoader
from demeterchain.splitters import TextSplitter

loader = TextLoader('wiki_datas/', glob='*.txt', show_progress=True)
documents = loader.load()

text_splitter = TextSplitter(separator="。",chunk_size=256, chunk_overlap=64)
documents = text_splitter.split_documents(documents)

100%|██████████| 12/12 [00:00<00:00, 21372.25it/s]
Created a chunk of size 474, which is longer than the specified 256


In [7]:
documents

[Document(page_content='蘋果樹（學名：Malus domestica）是薔薇科蘋果亞科蘋果屬植物，為落葉喬木，在世界上廣泛種...', metadata={'source': 'wiki_datas/蘋果.txt'}),
 Document(page_content='\n\n蘋果起源於中亞，直到今天當地還可以找到蘋果的野生祖先：新疆野蘋果。蘋果在亞洲和歐洲都有著數千年的...', metadata={'source': 'wiki_datas/蘋果.txt'}),
 Document(page_content='果實成長期之長短，一般早熟品種為65-87天，中熟品種為90-133天，晚熟品種則為137-168天...', metadata={'source': 'wiki_datas/蘋果.txt'}),
 Document(page_content='蘋果樹及其果實很容易出現許多真菌、細菌和害蟲問題，這些問題可以通過多種有機和非有機手段加以控制。 2...', metadata={'source': 'wiki_datas/蘋果.txt'}),
 Document(page_content='智慧型手機（英語：smartphone）是一種既可用來撥打移動電話具有多功能任務處理移動計算的行動裝...', metadata={'source': 'wiki_datas/智慧型手機.txt'}),
 Document(page_content='\n\n隨著行動網際網路發展，行動應用程式市場及行動商務、手機遊戲產業、社交即時通訊網路高度繁榮，產生了...', metadata={'source': 'wiki_datas/智慧型手機.txt'}),
 Document(page_content='香蕉（學名：Musa × paradisiaca），又名甘蕉、芎蕉、芽蕉，弓蕉（閩南語：king-t...', metadata={'source': 'wiki_datas/香蕉.txt'}),
 Document(page_content='主要生長於熱帶、亞熱帶地區。原產於亞洲東南部熱帶、亞熱帶地區。佤語稱為「muah nbaex」[2]...', metadata={'source': 'wiki_datas/香蕉.txt'}),
 Docum

# 設定retriever

In [3]:
# 安裝jdk-11，如果使用jupyter notebook時無法輸入密碼，請將以下命令複製到terminal執行
! sudo apt-get update
! sudo apt-get install openjdk-11-jdk

[sudo] password for 111_ms_wenchuan: 
[sudo] password for 111_ms_wenchuan: 

In [ ]:
# 安裝pyserini
! pip install --upgrade pip
! pip install faiss-cpu==1.7.4 pyserini==0.22.1

In [3]:
from demeterchain.retrievers import PyseriniBM25Retriever

retriever = PyseriniBM25Retriever.from_documents(documents, savepath="pyserini_bm25_index")

2024-06-18 16:37:09,434 INFO  [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO
2024-06-18 16:37:09,434 INFO  [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer...
2024-06-18 16:37:09,435 INFO  [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============
2024-06-18 16:37:09,435 INFO  [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: pyserini_bm25_index/docs
2024-06-18 16:37:09,435 INFO  [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection
2024-06-18 16:37:09,435 INFO  [main] index.IndexCollection (IndexCollection.java:387) - Generator: DefaultLuceneDocumentGenerator
2024-06-18 16:37:09,435 INFO  [main] index.IndexCollection (IndexCollection.java:388) - Threads: 1
2024-06-18 16:37:09,435 INFO  [main] index.IndexCollection (IndexCollection.java:389) - Language: zh
2024-06-18 16:37:09,435 INFO  [main] index.IndexCollectio

In [2]:
# 測試
retriever.invoke("甚麼水果產於南美洲", k=3)

[Document(page_content='鳳梨（Ananas comosus），俗名菠蘿、露兜子[1]，是原產於南美洲的熱帶水果，為禾本目鳳梨...', metadata={'source': 'wiki_datas/鳳梨.txt'}),
 Document(page_content='蘋果樹及其果實很容易出現許多真菌、細菌和害蟲問題，這些問題可以通過多種有機和非有機手段加以控制。 2...', metadata={'source': 'wiki_datas/蘋果.txt'}),
 Document(page_content='主要生長於熱帶、亞熱帶地區。原產於亞洲東南部熱帶、亞熱帶地區。佤語稱為「muah nbaex」[2]...', metadata={'source': 'wiki_datas/香蕉.txt'})]

In [3]:
# 此retriever使用java，不支援保存，只能在建立時保存，請勿使用此功能
retriever.save("bm25")

PyseriniBM25Retriever does not support save method, the index has been automatically saved in pyserini_bm25_index when built


In [1]:
# 讀取
from demeterchain.retrievers import PyseriniBM25Retriever

retriever = PyseriniBM25Retriever.load("pyserini_bm25_index")

[2024-05-14 07:15:36,870] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


# 設定回答模型

In [ ]:
! huggingface-cli login

In [5]:
model_name_or_path = "/user_data/demetergpt/model/taide_noanswer"#"NchuNLP/taide-qa" # 需要申請權限

In [6]:
from demeterchain.models import GenerativeModel
from demeterchain.utils import QAModelConfig, PromptTemplate


model_config = QAModelConfig(
    model_name_or_path = model_name_or_path,
    device_map = "auto",
    quantize = "bitsandbytes-nf4",
    noanswer_str = "無法回答。",
    template = PromptTemplate(
        input_variables = ["doc", "query"],
        template="[INST] <<SYS>>\n請根據提供的問題，從提供的內文中尋找答案並回答，回答時只需要輸出答案，不需輸出其他資訊，如果從提供的內文無法找到答案，請回答\"無法回答\"\n<</SYS>>\n\n問題:\n{query}\n\n內文:\n{doc}\n [/INST]答案:\n"
    )
)
reader = GenerativeModel(config=model_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [7]:
# 測試
from demeterchain.utils import Document

inputs = {"query": "什麼水果是紅色的?"}
docs = [
    "蘋果是紅色的水果",
    "香蕉是黃色的水果",
    "我是小明，今天天氣很好",
    "草莓不是黑的，是紅色的水果"]
docs = [Document(page_content=doc) for doc in docs]
answer_doc = reader.get_answer(inputs, docs)

print(answer_doc)

100%|██████████| 4/4 [00:02<00:00,  1.44it/s]

{'蘋果': Document(page_content='蘋果是紅色的水果', metadata={}), '草莓': Document(page_content='草莓不是黑的，是紅色的水果', metadata={})}


# 建立HyDE與最後摘要所要使用的LLM

In [13]:
# 請調整成自己需要的LLM
token = "your_token" # Please fill in your token
host = "https://api.openai.com/v1"
model = "gpt-4-turbo"

In [14]:
from demeterchain.utils import MessageTemplate
from demeterchain.llms import ChatCompletionsLLM

# https://arxiv.org/abs/2212.10496
hyde_template = MessageTemplate(
    input_variables = ["query"],
    template=[
        {"role": "system", "content": "請幫我解答以下問題，請用中文回答。",},
        {"role": "user", "content": "{query}"}
    ]
)
hyde_llm = ChatCompletionsLLM(
    host = host,
    model = model,
    token = token,
    template= hyde_template,
    generation_config={
        "max_tokens": 200,
        "temperature": 0.9,
    }
)

In [15]:
hyde_llm({"query": "甚麼水果是紅色的?"})

'紅色的水果有：櫻桃、櫻瓜、石榴、櫻桃、胡椒玫瑰、西瓜、李子、楓糖樹、青檸、芒果、蓮霧。'

In [16]:
from demeterchain.utils import MessageTemplate
from demeterchain.llms import ChatCompletionsLLM

summary_template = MessageTemplate(
    input_variables = ["query", "answers"],
    template=[
        {"role": "system", "content": "請幫完成任務以下問題，請用中文回答。",},
        {"role": "user", "content": "我有以下片段關於{query}的答案：{answers} 你可以試著幫我將結果進行統整摘要嗎？"}
    ]
)
summary_llm = ChatCompletionsLLM(
    host = host,
    model = model,
    token = token,
    template= summary_template,
    generation_config={
        "max_tokens": 600,
        "temperature": 0.9,
    }
)

In [18]:
summary_llm({"query": "甚麼水果是紅色的?", "answers": "櫻桃,櫻瓜,石榴"})

'根據所提供的片段，能歸納出以下結論：\n\n該片段提及的紅色水果有：櫻桃、櫻桃、石榴。 \n\n然而需要注意的是，雖然這些水果都帶有紅色，但各自的紅色深淺度不同，紅石榴的紅色較深，櫻桃則帶有亮麗的紅色，而櫻桃的紅色則偏向玫紅色。 \n\n此外，還需考量到有些水果在成熟過程中會從綠色轉變為紅色，例如一些型別的辣椒或番茄，因此僅根據提及的水果種類，無法確定它們是否都是始終帶有紅色的。 \n'

# 串接全部套件

In [7]:
from demeterchain.utils import QAConfig
from demeterchain.chains import RetrievalQA

qa = RetrievalQA(
    reader=reader, 
    retriever=retriever, 
    #hyde_llm=hyde_llm,
    #summary_llm=summary_llm
)
qa_config = QAConfig(retrieve_k = 3)

In [8]:
result = qa({"query": "甚麼水果產於南美洲?"}, qa_config=qa_config)

100%|██████████| 3/3 [00:22<00:00,  7.41s/it]


In [9]:
answers = [answer.answer for answer in result.answers]
print(f"摘要結果: {result.summary}")
print(f"預測答案: {answers}")

摘要結果: None
預測答案: ['鳳梨']


In [22]:
result = qa({"query": "恐龍好吃嗎?"}, qa_config=qa_config)
answers = [answer.answer for answer in result.answers]
print(answers)

100%|██████████| 3/3 [00:03<00:00,  1.15s/it]

['很抱歉，模型無法根據現有資料集回答您的問題。']
